In [3]:
import pandas as pd

# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [14]:
data = pd.read_csv('demo01.csv')

linha = '-' * 90

# As variáveis
print(data.head())
print(linha)

# Tipo de dados de cada variável
print(data.dtypes, '\n')
print(linha)

# Quantidade de Missings
print(data.isnull().sum(), '\n')
print(linha)

# Distribuição da Variável Resposta (Mau)
print(data['mau'].value_counts(), '\n')
print(linha)

  sexo posse_de_veiculo posse_de_imovel  qtd_filhos            tipo_renda  \
0    M                Y               Y           0               Working   
1    F                N               Y           0  Commercial associate   
2    F                N               Y           0  Commercial associate   
3    M                Y               Y           0               Working   
4    F                Y               N           0               Working   

                        educacao          estado_civil    tipo_residencia  \
0  Secondary / secondary special               Married  House / apartment   
1  Secondary / secondary special  Single / not married  House / apartment   
2  Secondary / secondary special  Single / not married  House / apartment   
3               Higher education               Married  House / apartment   
4              Incomplete higher               Married  House / apartment   

       idade  tempo_emprego  possui_celular  possui_fone_comercial  \
0  5

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável. 
    Dica: 
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis, 
        3. conte o número de categorias dessa variável 
        4. acumule essa informação de 3. na lista que você criou em 1. 
        5. No final, essa lista pode ser atribuída à nossa variável.
3. Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}.

In [16]:
metadata = pd.DataFrame({'Variável': data.columns, 'Tipo de Dados': data.dtypes})

categorias = []
for col in data.columns:
    if data[col].dtype == 'object':
        categorias.append(data[col].nunique())
    else:
        categorias.append(0)
        
metadata['qtd_categorias'] = categorias

qualitativas = metadata[metadata['Tipo de Dados'] == 'object']['Variável'].tolist()
dummy_variables = pd.get_dummies(data[qualitativas])

print(metadata)

print(dummy_variables.head())

                                    Variável Tipo de Dados  qtd_categorias
sexo                                    sexo        object               2
posse_de_veiculo            posse_de_veiculo        object               2
posse_de_imovel              posse_de_imovel        object               2
qtd_filhos                        qtd_filhos         int64               0
tipo_renda                        tipo_renda        object               5
educacao                            educacao        object               5
estado_civil                    estado_civil        object               5
tipo_residencia              tipo_residencia        object               6
idade                                  idade       float64               0
tempo_emprego                  tempo_emprego       float64               0
possui_celular                possui_celular         int64               0
possui_fone_comercial  possui_fone_comercial         int64               0
possui_fone              

#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [17]:
qualitativas = metadata[(metadata['Tipo de Dados'] == 'object') & (metadata['qtd_categorias'] > 2)]['Variável'].tolist()
dummy_variables = pd.get_dummies(data[qualitativas])

numericas = metadata[metadata['Tipo de Dados'] != 'object']['Variável'].tolist()
numeric_data = data[numericas]

processed_data = pd.concat([numeric_data, dummy_variables], axis=1)

print(processed_data.head())

   qtd_filhos      idade  tempo_emprego  possui_celular  \
0           0  58.832877       3.106849               1   
1           0  52.356164       8.358904               1   
2           0  52.356164       8.358904               1   
3           0  46.224658       2.106849               1   
4           0  29.230137       3.021918               1   

   possui_fone_comercial  possui_fone  possui_email  qt_pessoas_residencia  \
0                      0            0             0                    2.0   
1                      0            1             1                    1.0   
2                      0            1             1                    1.0   
3                      1            1             1                    2.0   
4                      0            0             0                    2.0   

     mau  tipo_renda_Commercial associate  ...  estado_civil_Married  \
0  False                                0  ...                     1   
1  False                        

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [18]:
tabela_cruzada = pd.crosstab(data['possui_email'], data['posse_de_veiculo'])

print(tabela_cruzada)

posse_de_veiculo     N     Y
possui_email                
0                 9341  5829
1                  837   643


**Resposta:** A variável [possui_email] tem 837 casos em que 'mau' é igual a 1, contra 643 casos da variável [posse_de_veiculo], sendo assim podemos inferir que a variável [possui_email] parece ser mais poderosa para prever a probabilidade solicitada.

#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [19]:
processed_data.to_csv('base_processada.csv', index=False)